In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from matplotlib import colors
import networkx as nx

In [ ]:
df1 = pd.read_excel('T18C.xlsx')
df2 = pd.read_excel('T115C.xlsx')
df3 = pd.read_excel('T122C.xlsx')
df4 = pd.read_excel('T28C.xlsx')
df5 = pd.read_excel('T215C.xlsx')
df6 = pd.read_excel('T222C.xlsx')

In [ ]:
combined_data2 = [df4, df5, df6]
combined_data1 = [df1, df2, df3]
dft2 = pd.concat(combined_data2, axis=0, ignore_index=True)
dft1 = pd.concat(combined_data1, axis=0, ignore_index=True)
df = dft2 - dft1


In [ ]:
print(df)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
columns = df.columns
spearman_corr_matrix = pd.DataFrame(index=columns, columns=columns)
p_values_matrix = pd.DataFrame(index=columns, columns=columns)

for i in range(len(columns)):
    for j in range(i, len(columns)):
        col1, col2 = columns[i], columns[j]
        spearman_corr, p_value = spearmanr(df[col1], df[col2])
        spearman_corr_matrix.loc[col1, col2] = spearman_corr
        spearman_corr_matrix.loc[col2, col1] = spearman_corr
        p_values_matrix.loc[col1, col2] = p_value
        p_values_matrix.loc[col2, col1] = p_value

In [ ]:
spearman_corr_matrix = spearman_corr_matrix.apply(pd.to_numeric, errors='coerce')
#plt.figure(figsize=(60, 30))
#sns.heatmap(spearman_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
#plt.title('Spearman Correlation Matrix')
#plt.show()

In [ ]:
p_values_matrix = p_values_matrix.apply(pd.to_numeric, errors='coerce')
#plt.figure(figsize=(60, 30))
#sns.heatmap( p_values_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
#plt.title('P_values_matrix Correlation Matrix')
#plt.show()

In [ ]:
alpha=0.05
p_values_flat = p_values_matrix.values.flatten()
rejected, corrected_p_values, _, alpha_corrected = multipletests(p_values_flat, alpha=alpha, method='bonferroni')
print(np.sum(rejected))
print(alpha_corrected)

In [ ]:
corrected_p_values_matrix = pd.DataFrame(corrected_p_values.reshape(p_values_matrix.shape), 
                                         index=columns, columns=columns)
#plt.figure(figsize=(60, 30))
#sns.heatmap(corrected_p_values_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
#plt.title('P_values_matrix Correlation Matrix')
#plt.show()


In [ ]:
significance_level = 0.05
significant_correlations = np.abs(p_values_matrix) < significance_level 

# Select only the upper triangle of the correlation matrix to avoid duplicates
upper_triangle = np.triu(np.ones(p_values_matrix.shape), k=1)

# Apply the significance filter and upper triangle mask
significant_upper_triangle = np.logical_and(significant_correlations, upper_triangle)

# Extract the indices of significant correlations
result_indices = np.where(significant_upper_triangle)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Variable1': spearman_corr_matrix.columns[result_indices[1]],
    'Variable2': spearman_corr_matrix.columns[result_indices[0]],
    'Correlation': spearman_corr_matrix.values[result_indices],
    'P-Value': p_values_matrix.values[result_indices]
})

sorted_results = results.sort_values(by='Correlation')

# Filter values that are greater than or equal to 0.5 or less than or equal to -0.5
significant_results = sorted_results[(sorted_results['Correlation'] >= 0.5) | (sorted_results['Correlation'] <= -0.5)]
# Display the significant results
print(significant_results)

In [ ]:
duplicates_all_columns = significant_results[significant_results.duplicated(keep=False)]
print("Rows with Duplicates Across All Columns:")
print(duplicates_all_columns)


In [ ]:
#!pip install holoviews


# Correlations between changes in cytokines and metabolites (Time point 2 minus time point 1)


In [ ]:
cytokines = list(df.columns[0:29])

In [ ]:
metabolites = list(df.columns[29:113])

In [ ]:
mask = significant_results['Variable1'].isin(metabolites) & significant_results['Variable2'].isin(cytokines)

# Use the mask to select rows containing both 'variable 1' and 'variable 2' in the metabolites list
filtered_df = significant_results[mask]

# Display the filtered DataFrame
print(filtered_df)

In [ ]:
len(filtered_df['Correlation'])

In [ ]:
corr = {
    'Source': ['1,5-Anhydrohexitol',
 '1,5-Anhydrohexitol',
 '1,5-Anhydrohexitol',
 '1,5-Anhydrohexitol',
 'Cholesterol',
 'Cholesterol',
 'Ethanolamine',
 '1,3-Dioxolane-2-methanol',
 '1,3-Dioxolane-2-methanol',
 '1,5-Anhydrohexitol',
 'Dodecanoic acid',
 'Tyrosine',
 'L-Rhamnose',
 'Glutamine [-H2O]'],
 
    'Target':  ['sCD30/TNFRSF8',
 'sCD163',
 'sIL-6Ra',
 'Chitinase 3-like1',
 'Osteopontin (OPN)',
 'sTNF-R2',
 'sTNF-R2',
 'IFN-a2',
 'IL-26',
 'BAFF/TNFSF13B',
 'sTNF-R1',
 'APRIL/TNFSF13',
 'MMP-1',
 'IL-12(p40)'],

    'Correlation': [0.636364, 0.692073, 0.705170, 0.729487, 0.762209, 0.766871, 0.780260, 0.819576, 0.820672, 1,1,1,1,1]
}


links_df = pd.DataFrame(corr)


In [ ]:
import holoviews as hv
from holoviews import opts, dim
import holoviews.plotting.bokeh
hv.extension('bokeh')
hv.output(size=250)

# Sample nodes data
nodes = ['1,5-Anhydrohexitol',
 '1,5-Anhydrohexitol',
 '1,5-Anhydrohexitol',
 '1,5-Anhydrohexitol',
 'Cholesterol',
 'Cholesterol',
 'Ethanolamine',
 '1,3-Dioxolane-2-methanol',
 '1,3-Dioxolane-2-methanol',
 '1,5-Anhydrohexitol',
 'Dodecanoic acid',
 'Tyrosine',
 'L-Rhamnose',
 'Glutamine [-H2O]',
 'sCD30/TNFRSF8',
 'sCD163',
 'sIL-6Ra',
 'Chitinase 3-like1',
 'Osteopontin (OPN)',
 'sTNF-R2',
 'sTNF-R2',
 'IFN-a2',
 'IL-26',
 'BAFF/TNFSF13B',
 'sTNF-R1',
 'APRIL/TNFSF13',
 'MMP-1',
 'IL-12(p40)']
nodes_df = pd.DataFrame(nodes)
nodes_df.rename(columns={nodes_df.columns[0]: 'node'}, inplace=True)
node = hv.Dataset(pd.DataFrame(nodes_df['node']))

# Create HoloViews Chord Diagram
chord = hv.Chord((links_df, node)).select(value=(None, None))

#custom_colormap = {True: 'blue', False: 'red'}
#links_df['Positive'] = links_df['Value'] > 0
#edge_colors = links_df['Positive'].map(custom_colormap)


# Customize chart options
chord.opts(opts.Chord(cmap='Category20', 
                edge_cmap='Category20',
                edge_color=dim('Source').str(),
                labels='node',
                edge_line_width=4,
                node_color=dim('node').str(),
                label_text_font_size='12pt'))
